# Removing Duplicate Students From Foot Traffic

In February 2019, I created an electronic sign-in sheet for the academic advising office foot traffic and launched it. For decades, the signin sheet was a paper form that recorded all the students who walked into the office, what they came in for, and the type of student they were. When I started data analysis, I transfered tens of thousands of students from the paper forms into electronic form in Excel. This was the only way to have the data be useful. Prior to my initiative, the data had never been aggregated by the college. It was recorded but not used to its fullest potential. The electronic signin sheet, obviously made my analysis much easier.

A separate algorithm sets the data up to be analyized. That is the csv that is referenced in the beginning--"Dashboard Setup.csv." One of the things the Director of Advising wanted to track was the workload of each advisor. As I dug into the data, it became clear very quickly that advisors had different entry patterns. Some would enter a student every time they interacted with them. For instance, an advisor could enter a students a half a dozen or a dozen times in a single week if they had multiple emails with the student. To address this, I wrote this algorithm to filter out duplicate students so that we could more accurately assess the number of students seen by each advisor. 

In [2]:
import pandas as pd
import numpy as np

import re
import sklearn

from pathlib import Path
import os
import sys

# Create path to code folder
CODE_FOLDER = Path('code')
CODE_FOLDER.mkdir(exist_ok=True)
sys.path.extend([f"./{CODE_FOLDER}"])

# Create path to data
DATA_PATH = Path.cwd()/'Files'

In [4]:
from utilities import retrieve_and_open_csv_files

# Import data
adv_raw = retrieve_and_open_csv_files(DATA_PATH, keyword='Dashboard')

# Change date to datetime format
adv_raw['DATE'] = pd.to_datetime(adv_raw['DATE'], format = 'mixed')

# Add the week
adv_raw['WEEK'] = adv_raw['DATE'].dt.isocalendar().week

In [5]:
# Calculate the number of dupilcate names in the foot traffic data.
len(adv_raw) - len(adv_raw.drop_duplicates(['MONTH', 'NAME', 'ADVISOR SIGN']))

3353

In [6]:
#This dataframe shows the counts of duplicates per month by advisor without
#any additional filtering
(pd.DataFrame(adv_raw[adv_raw.duplicated(['MONTH', 'NAME', 'ADVISOR SIGN'])]
    .groupby('ADVISOR SIGN')['ID'].count()).rename(columns = {'ID':'COUNT'})
    .reset_index()
    .sort_values('COUNT', ascending = False)
    .reset_index(drop = True)
)

,ADVISOR SIGN,COUNT
0,RM,656
1,BM,432
2,SV,308
3,BG,279
4,KLA,267
5,SH,228
6,DR,223
7,SLB,217
8,KZ,174
9,TB,105


In [7]:
#This dataframe we take into account the 'distance' column. We isolate just 
#the instances in which there are no null values in the row. Null values
#are either because there were appointments or because they were walkins.
#This leaves us with a dataframe of 14705 interactions.

#Out of that 14705, 2326 were duplicates. So these duplicates reflect
#instances in which the advisor chose to keep entering the student when
#they called or emailed them. Notice that some of your lower people who
#had duplicates (those <= 20), drop completely off or down to low single 
#digits. 
mask = adv_raw['DISTANCE'].isnull() == False

df2 = adv_raw[mask]

adv_dups = (pd.DataFrame(df2[df2.duplicated(['MONTH', 'NAME', 'ADVISOR SIGN'])]
             .groupby('ADVISOR SIGN')['ID'].count()).rename(columns = {'ID':'COUNT'})
             .reset_index().sort_values('COUNT', ascending = False)
             .reset_index(drop = True)
           )

adv_dups

,ADVISOR SIGN,COUNT
0,RM,557
1,BM,348
2,BG,234
3,KLA,179
4,SV,179
5,DR,163
6,SLB,156
7,SH,139
8,KZ,115
9,TB,60


In [8]:
#Below is a dataframe with all students an advisor assisted throughout a 
#year REGARDLESS OF DUPLICATION. 
all_adv = (pd.DataFrame(adv_raw.groupby('ADVISOR SIGN')['ID'].count())
            .reset_index().rename(columns = {'ID':'COUNT'})
            .sort_values('COUNT', ascending = False)
            .reset_index(drop = True)
          )

all_adv

,ADVISOR SIGN,COUNT
0,BM,3840
1,RM,3747
2,SH,2889
3,DR,2234
4,BG,2089
5,KZ,2027
6,KLA,1894
7,SV,1772
8,SLB,1607
9,AKP,1417


In [9]:
#Now we merge all of the signatures with the duplicated signatures
all_and_dups = (all_adv.merge(adv_dups, how = 'left', on = 'ADVISOR SIGN')
                       .rename(columns = {'COUNT_y':'DUPLICATED',
                                         'COUNT_x':'FULL_COUNT'})
                       .fillna(0)
               )

#clean up the dataframe to eliminate ['??', 'LEFT', '8:03']
all_and_dups = all_and_dups[all_and_dups['ADVISOR SIGN'].isin(['??', 'LEFT', '8:03']) == False]

#Calculate percent of the signatures from each advisor that are duplicated
#students
all_and_dups['PERC_OF_STUDENTS'] = round((all_and_dups['DUPLICATED']/all_and_dups['FULL_COUNT']) * 100, 3)

#Sort from highest to lowest of percent of students that are duplicated 
#by each advisor
all_and_dups = (all_and_dups.sort_values('PERC_OF_STUDENTS', ascending = False).reset_index(drop = True)
                            .reset_index().rename(columns = {'index':'PERC_RANK'})
                            .sort_values('FULL_COUNT', ascending = False).reset_index(drop = True)
               )
all_and_dups

,PERC_RANK,ADVISOR SIGN,FULL_COUNT,DUPLICATED,PERC_OF_STUDENTS
0,6,BM,3840,348.0,9.062
1,0,RM,3747,557.0,14.865
2,11,SH,2889,139.0,4.811
3,8,DR,2234,163.0,7.296
4,1,BG,2089,234.0,11.202
5,10,KZ,2027,115.0,5.673
6,4,KLA,1894,179.0,9.451
7,2,SV,1772,179.0,10.102
8,3,SLB,1607,156.0,9.708
9,18,AKP,1417,10.0,0.706


In [10]:
#Only run through this cell once. Each time you run it, it will 
#add another to the ranking
all_and_dups['PERC_RANK'] = all_and_dups['PERC_RANK'] + 1

In [11]:
all_and_dups = all_and_dups.reset_index().rename(columns = {'index':'RANK'})

all_and_dups['RANK'] = all_and_dups['RANK'] + 1

One of the revelations of this analysis is that the advisor who assisted the second highest number of students, when considering the raw count, had 15% of the students entered as duplicates, which draws that number way down. The second consideration is that advisors who oversee the advising email account will, by default, have far more students in their count than advisors who do not have access to that account. 

This analysis influenced the Director of Advising to put every advisor in the college in the advising email account because the workload was so drastically imbalanced. 

In [16]:
(all_and_dups
     .sort_values('PERC_RANK')
     .reset_index(drop = True)
)

,RANK,PERC_RANK,ADVISOR SIGN,FULL_COUNT,DUPLICATED,PERC_OF_STUDENTS
0,2,1,RM,3747,557.0,14.865
1,5,2,BG,2089,234.0,11.202
2,8,3,SV,1772,179.0,10.102
3,9,4,SLB,1607,156.0,9.708
4,7,5,KLA,1894,179.0,9.451
5,17,6,JEC,545,51.0,9.358
6,1,7,BM,3840,348.0,9.062
7,15,8,TB,748,60.0,8.021
8,4,9,DR,2234,163.0,7.296
9,22,10,PP,180,11.0,6.111
